# artsy

In [1]:
import json

from geopy.geocoders import Nominatim
from pymongo import MongoClient

## setup

### database

In [2]:
client = MongoClient()
# client.drop_database('artsy')
db = client.artsy

### geolocator

In [3]:
geolocator = Nominatim()

In [4]:
geolocator.geocode('Seville').latitude

37.3886303

## Data

In [5]:
data = []
count = 0
for artist in db.artists.find():
    location = artist['location'] or artist['hometown']
    if location:
        artist_id = artist['_id']
        artists_artworks = db.artists_artworks.find_one({'_id': artist_id})
        if artists_artworks:
            artwork_ids = artists_artworks['data']
            for artwork_id in artwork_ids:
                artworks_genes = db.artworks_genes.find_one({'_id': artwork_id})
                count += 1
count

9497

In [11]:
db.geolocation.count()

8252

In [28]:
db.geolocation2.count()

9077

In [26]:
%%time
db.geolocation2.drop()
total_count = db.artists.count()
size = 1000
c=0
chunks = [db.artists.find()[x:x+size] for x in range(0, total_count, size)]
for chunk in chunks:
    data = []
    for artist in chunk:
        c+=1
        artist_id = artist['_id']
        location = artist['location'] or artist['hometown']
        if location:
            artists_artworks = db.artists_artworks.find_one({'_id': artist_id})
            if artists_artworks:
                artwork_ids = artists_artworks['data']
                if artwork_ids:
                    try:
                        geo = geolocator.geocode(location)
                        if geo:
                            for artwork_id in artwork_ids:
                                artworks_genes = db.artworks_genes.find_one({'_id': artwork_id})
                                genes = []
                                if artworks_genes:
                                    gene_ids = artworks_genes['data']
                                    for gene_id in gene_ids:
                                        gene = db.genes.find_one({'_id': gene_id})
                                        if gene:
                                            genes.append(gene['name'])
                                info = {
                                    '_id': artwork_id,
                                    'latitude': geo.latitude,
                                    'longitude': geo.longitude,
                                    'genes': genes,
                                }
                                data.append(info)
                    except:
                        pass
        if c%1000==0:
            print(c)
    db.geolocation2.insert_many(data)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
CPU times: user 32.3 s, sys: 3.13 s, total: 35.4 s
Wall time: 6min 37s


In [10]:
%%time
# db.geolocation.drop()
total_count = db.artists.count()
size = 1000
c=0
chunks = [db.artists.find()[x:x+size] for x in range(0, total_count, size)]
for chunk in chunks:
    data = []
    for artist in chunk:
        c+=1
        artist_id = artist['_id']
        location = artist['location'] or artist['hometown']
        if location:
            artists_artworks = db.artists_artworks.find_one({'_id': artist_id})
            if artists_artworks:
                artwork_ids = artists_artworks['data']
                if artwork_ids:
                    geo = geolocator.geocode(location)
                    if geo:
                        for artwork_id in artwork_ids:
                            artworks_genes = db.artworks_genes.find_one({'_id': artwork_id})
                            genes = []
                            if artworks_genes:
                                gene_ids = artworks_genes['data']
                                for gene_id in gene_ids:
                                    gene = db.genes.find_one({'_id': gene_id})
                                    if gene:
                                        genes.append(gene['name'])
                            info = {
                                '_id': artwork_id,
                                'latitude': geo.latitude,
                                'longitude': geo.longitude,
                                'genes': genes,
                            }
                            data.append(info)
        if c%1000==0:
            print(c)
    db.geolocation.insert_many(data)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


GeocoderTimedOut: Service timed out

In [29]:
artworks = []
artworks_ids = set()
genes = []
genes_ids = set()
relationships = set()
for geo in db.geolocation2.find():
    artwork_node = {
        'artwork_id': geo['_id'],
        'latitude': geo['latitude'],
        'longitude': geo['longitude'],
    }
    
    if geo['_id'] not in artworks_ids:
        artworks.append(artwork_node)
    artworks_ids.add(geo['_id'])
    
    for gene in geo['genes']:
        gene_node = {
            'name': gene,
        }
        
        if gene not in genes_ids:
            genes.append(gene_node)
        genes_ids.add(gene)
        
        relationships.add((geo['_id'], gene))
relationships = list(relationships)

In [30]:
len(relationships)

35084

In [31]:
js = {
    'artwork_nodes': artworks,
    'gene_nodes': genes,
    'relationships': relationships,
}

In [33]:
import json
with open('artsy2.json', 'w') as outfile:
    json.dump(js, outfile)